<a href="https://colab.research.google.com/github/Minwooh/dalc-mini/blob/master/minwoo_hw2_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TA-Lib 설치 코드
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3
import talib

# 야후 파이낸스설치
!pip install yfinance
import yfinance as yf

import pandas as pd
import numpy as np

In [ ]:
# 하이브 데이터 불러오기
hive_df= df = yf.download('352820.KS',start = '2005-01-01') # 우리나라원하는 주식의 종목코드+.KS
sox_df = yf.download('^SOX', start = '2000-01-01')
vix_df = yf.download('^VIX', start = '2000-01-01')
snp500_df = yf.download('^GSPC', start = '2000-01-01')
kospi_df= yf.download('^KS11', start = '2000-01-01')

kospi_df.to_csv('kospi.csv')
hive_df.to_csv('hive.csv')
sox_df.to_csv('sox_df.csv')
vix_df.to_csv('vix_df.csv')
snp500_df.to_csv('s&p500.csv')

hive_df = pd.read_csv('hive.csv',index_col='Date', parse_dates=True)
sox_df = pd.read_csv('sox_df.csv', index_col = 'Date', parse_dates = True)
vix_df = pd.read_csv('vix_df.csv', index_col = 'Date', parse_dates = True)
snp500_df = pd.read_csv('s&p500.csv', index_col = 'Date', parse_dates = True)
kospi_df = pd.read_csv('kospi.csv', index_col = 'Date', parse_dates = True)

In [ ]:
snp500_df = snp500_df.loc[:,['Close']].copy()
snp500_df.rename(columns={'Close':'S&P500'},inplace=True)
sox_df = sox_df.loc[:,['Close']].copy()
sox_df.rename(columns={'Close':'SOX'},inplace=True)
vix_df = vix_df.loc[:,['Close']].copy()
vix_df.rename(columns={'Close':'VIX'},inplace=True)


df = df.join(snp500_df,how='left')
df = df.join(sox_df,how='left')
df = df.join(vix_df,how='left')

In [ ]:
kospi_df['KOSPI'] = kospi_df['Close'].copy()

In [ ]:
kospi_df['kosATR'] = talib.ATR(kospi_df['High'], kospi_df['Low'], kospi_df['Close'], timeperiod=14)

kospi_df['kosADX14'] = talib.ADXR(kospi_df['High'], kospi_df['Low'], kospi_df['Close'], timeperiod=14)


kospi_df = kospi_df.drop(['Open','High','Low','Close','Adj Close','Volume'],axis=1)
df = df.join(kospi_df, how = 'left')

In [ ]:
df

In [ ]:
# 기술지표 가공
df['log_return'] = np.log(1 + df['Adj Close'].pct_change())
df['CCI'] = talib.CCI(df['High'], df['Low'], df['Adj Close'], timeperiod=14)


#1.RA : Standard deviation rolling average
# Moving Average
df['MA10'] = talib.SMA(df['Close'],timeperiod=10)
df['MA60'] = talib.SMA(df['Close'],timeperiod=60)

df['MA60_volumne'] = talib.SMA(df['Volume'], timeperiod = 60)
df['RASD5'] = talib.SMA(talib.STDDEV(df['Close'], timeperiod=5, nbdev=1),timeperiod=5)
df['RASD10'] = talib.SMA(talib.STDDEV(df['Close'], timeperiod=5, nbdev=1),timeperiod=10)

#2.MACD : Moving Average Convergence/Divergence
macd, macdsignal, macdhist = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['MACD'] = macd 

# Momentum Indicators
#3.CCI : Commodity Channel Index

df['CCI60'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=60)
df['CCI60'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=90)
# Volatility Indicators 

#4.ATR : Average True Range
df['ATR14'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=14)

#7.MTM
df['MTM3'] = talib.MOM(df['Close'], timeperiod=3)

#8.ROC : Rate of change : ((price/prevPrice)-1)*100
df['ROC60'] = talib.ROC(df['Close'], timeperiod=60)
df['ROC90'] = talib.ROC(df['Close'], timeperiod=30)

#9.WPR : william percent range (Williams' %R)
df['WPR14'] = talib.WILLR(df['High'], df['Low'], df['Close'], timeperiod=14)

#10.ADXR
df['ADXR14'] = talib.ADXR(df['High'], df['Low'], df['Close'], timeperiod=14)

#11.ADX
df['ADX14'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=14)

In [ ]:
df.head()

In [ ]:
df.to_csv('352820.csv') #경로 지정

In [ ]:
df.head()

In [ ]:
df.info()